## Preprocessing:

Dada la precencia de __NaN__ llegada esta instancia y teniendo todos los datos _limpios_, podemos dedicarnos a solucionar este proble.
Lo primero que se hace es completar los datos faltantes segun el promedio de su zona para su tipo de propiedad, en caso de que esto no sea posible, se ira a la siguente zona mas abarcativa.
Estas zonas son, en primer lugar _place_name_with_parents_ que resulta ser bastante precisa cuando uno se aleja de capital federal, mientras que la segunda, es la divicion de Norte Sur Oeste y CABA, la cual es usada una cantidad mucho menor de veces que la primera zona.
Luego de completar todos los datos, se escalan los valores usando RobustScaler, que tiene la particularidad de ser "resistente" a los valores fuera de normal, algo bastante obversado en el set de datos.

In [12]:
import pandas as pd
import numpy as np

In [13]:
FILE_TRAIN = 'data/training-with-features.csv'
FILE_TEST = 'data/testing-with-features.csv'

In [14]:
drop = ['description','id']

In [15]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing
def f_robust_scale(df):
    headers = list(df)
    M = df.as_matrix()
    rs = RobustScaler()
    M = rs.fit_transform(M)
    df = pd.DataFrame(M)
    df.columns = headers
    return df

### Completando informacion faltante:

In [16]:
def completeNans(row, column, zona_1, zona_2, nivel_1, nivel_2):
    if np.isnan(row[column]):
        a = row[nivel_1]
        b = row['property_type']
        try:
            value = zona_1.loc[int(a),int(b)][column]
        except (KeyError, TypeError) as e:
            a = row[nivel_2]
            try:
                value = zona_2.loc[int(a),int(b)][column]
            except (KeyError, TypeError) as e:
                return 0
        if np.isnan(value):
            return 0
        return value
    return row[column]

In [17]:
df = pd.read_csv(FILE_TRAIN, low_memory=False)
# df = df.drop(df[['Unnamed: 0']], axis=1)
x = df.groupby(['place_name','property_type']).mean()
x = x.reset_index()
y = df.groupby(['state_name','property_type']).mean()
for column in list(y):
      x[column] = x.apply(lambda row: completeNans(row, column, y, None, 'state_name', None), axis=1)
x['place_name'] = x['place_name'].astype(int)
x['property_type'] = x['property_type'].astype(int)
x = x.set_index(['place_name','property_type'])

# Train DataSet

In [ ]:
df = pd.read_csv(FILE_TRAIN, low_memory=False)
prices = df['price']
df = df.drop(df[drop+['price']], axis=1)
x_train = x.drop(x[['price']], axis=1)
for column in list(x_train):
    if df[column].isnull().any() != 0:
        df[column] = df.apply(lambda row: completeNans(row, column, x_train, y,'place_name','state_name' ), axis=1)
df = f_robust_scale(df)
df_m = df.join(prices)
df_m.isnull().values.any()

In [ ]:
df_m.to_csv('data/pp-training.csv', index=False)

# Test DataSet

In [11]:
df = pd.read_csv(FILE_TEST, low_memory=False)
df = df.drop(df[drop], axis=1)
x_train = x.drop(x[['price']], axis=1)
for column in list(x_train):
    if df[column].isnull().any() != 0:
        df[column] = df.apply(lambda row: completeNans(row, column, x_train, y,'place_name','state_name' ), axis=1)
df = f_robust_scale(df)
df.isnull().values.any()

KeyError: 'Unnamed: 0'

In [10]:
df.to_csv('data/pp-testing.csv', index=False)